In [ ]:
# required depdendencies - build-essential python 3.12 python3.12-dev
%pip install setuptools autodistill autodistill-grounded-sam autodistill-yolov8 roboflow

In [ ]:
from autodistill_grounded_sam import GroundedSAM
from autodistill.detection import CaptionOntology
from autodistill_yolov8 import YOLOv8

# define an ontology to map class names to our GroundingDINO prompt
# the ontology dictionary has the format {caption: class}
# where caption is the prompt sent to the base model, and class is the label that will
# be saved for that caption in the generated annotations
base_model = GroundedSAM(ontology=CaptionOntology({"rabbit": "rabbit"}))


In [ ]:

# label all images in a folder called `context_images` using GroundingSAM
base_model.label(
  input_folder="./images",
  output_folder="./labeled_data"
)

In [ ]:
# Train yolov8n pre-trained model with autolabeled data
target_model = YOLOv8("yolov8n.pt")
target_model.train("./labeled_data/data.yaml", epochs=200)

In [ ]:
# (optional) Load pre-trained model with autolabeled data
from autodistill_yolov8 import YOLOv8Base
from autodistill.detection import CaptionOntology

target_model = YOLOv8Base(ontology=CaptionOntology({"rabbit": "rabbit"}), weights_path="./runs/detect/train/weights/best.pt")
print(target_model.ontology.classes())


In [ ]:
# Test trained model against an image

from autodistill.utils import plot
import cv2
image_path = "./animals.v3i/valid/images/Image_58_jpg.rf.5db70d270195e43b09df0b4a61548df3.jpg"
pred = target_model.predict(image_path)

print(pred)
plot(
    image=cv2.imread(image_path),
    detections=pred,
    classes="rabbit"
)

In [ ]:
# optional: upload your model to Roboflow for deployment
from roboflow import Roboflow

rf = Roboflow(api_key="API_KEY")
project = rf.workspace().project("animals-7fizt")
project.version(3).deploy(model_type="yolov8", model_path=f"./runs/detect/train/")

